In [1]:
import os
import time
import csv
import numpy as np
from sklearn.decomposition import NMF
from sonnmf.main import sonnmf
from sonnmf.old.main import sonnmf as sonnmf_old
import matplotlib.pyplot as plt

In [8]:
def sample_dirichlet(alpha, N):
    k = len(alpha)
    theta = np.zeros((N, k))
    scale = 1
    for i in range(k):
        theta[:, i] = np.random.gamma(alpha[i], scale, N)
    S = np.sum(theta, axis=1)
    theta = theta / np.tile(S.reshape(-1, 1), (1, k))
    return theta


def create_synthetic_data(n):
    Wt = np.array([[1, 0, 0, 1], [1, 0, 1, 0], [0, 1, 1, 0], [0, 1, 0, 1]])
    r = 4

    # n = 500
    purity = 0.8
    alpha = 0.05 * np.ones((r, 1))
    Ht = sample_dirichlet(alpha, n).T
    for j in range(n):
        while np.max(Ht[:, j]) > purity:
            Ht[:, j: j+1] = sample_dirichlet(alpha, 1).T
    epsilon = 0.01
    Xt = np.dot(Wt, Ht)
    X = np.maximum(0, Xt + epsilon * np.random.randn(*Xt.shape))
    return X, Wt, Ht


def plot_3d(X, Wt, W, filepath=None):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(X[0, :], X[1, :], X[2, :], c='lightblue', marker='o')
    ax.scatter(Wt[0, :], Wt[1, :], Wt[2, :], c='red', marker='o', s=144)
    ax.scatter(W[0, :], W[1, :], W[2, :], c='black', marker='x', s=144)
    ax.set_xlabel('X1')
    ax.set_ylabel('X2')
    ax.set_zlabel('X3')
    ax.legend(['Data points', 'True W', 'Estimated W'])
    ax.grid(True)
    plt.tight_layout()
    if filepath:
        plt.savefig(filepath)
    else:
        plt.show()
    plt.close()

def save_results(filepath, W, H, fscores, gscores, hscores, total_scores):
    with open(filepath, 'wb') as fout:
        np.savez_compressed(fout, W=W, H=H, fscores=fscores, gscores=gscores, hscores=hscores, total_scores=total_scores)

def load_results(filepath):
    data = np.load(filepath)
    return data['W'], data['H'], data['fscores'], data['gscores'], data['hscores'], data['total_scores']

def save_results_old(filepath, W, H, fscores, gscores, hscores, total_scores):
    with open(filepath, 'wb') as fout:
        np.savez_compressed(fout, W=W, H=H, fscores=fscores, gscores=gscores, total_scores=total_scores)

def load_results_old(filepath):
    data = np.load(filepath)
    return data['W'], data['H'], data['fscores'], data['gscores'], data['total_scores']

In [3]:
data_filepath = '../datasets/synthetic_n{}.npz'
ini_filepath = '../saved_models/multi_synthetic/n{}_r{}_ini.npz'
csv_filename = '../saved_models/multi_synthetic/results.csv'

In [4]:
max_iters = 10000
r = 8

In [5]:
# with open(csv_filename, 'a', newline='') as csv_file:
#     csv_writer = csv.writer(csv_file)
#     csv_writer.writerow(['Method', 'n', 'r', 'l', 'g', 'Iterations', 'Time Taken', 'Save Path', 'Image Path'])

In [6]:
save_filepath = '../saved_models/multi_synthetic/prox_avg_pen/n{}_r{}_l{}_g{}_it{}.npz'
image_filepath = '../images/multi_synthetic/prox_avg_pen/n{}_r{}_l{}_g{}_it{}.jpg'

for n in [10, 100, 1000, 10000]:

    if os.path.exists(data_filepath.format(n)):
        data = np.load(data_filepath.format(n))
        M = data['M']
        W_true = data['W_true']
        H_true = data['H_true']
    else:
        M, W_true, H_true = create_synthetic_data(n)
        with open(data_filepath.format(n), 'wb') as fout:
            np.savez_compressed(fout, M=M, W_true=W_true, H_true=H_true)

    m, n = M.shape
    if os.path.exists(ini_filepath.format(n, r)):
        data = np.load(ini_filepath.format(n, r))
        ini_W = data['ini_W']
        ini_H = data['ini_H']
    else:
        ini_W = np.random.rand(m, r) * 5
        ini_H = np.random.rand(r, n)
        with open(ini_filepath.format(n, r), 'wb') as fout:
            np.savez_compressed(fout, ini_W=ini_W, ini_H=ini_H)


    for g in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]:
        for l in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]:
            if not os.path.exists(save_filepath.format(n, r, l, g, max_iters)):
                start_time = time.time()
                W, H, fscores, gscores, hscores, total_scores = sonnmf(M, ini_W.copy(), ini_H.copy(), lam=l, gamma=g, itermax=max_iters, W_update_iters=10, early_stop=True, verbose=False)
                time_taken = time.time() - start_time

                save_results(save_filepath.format(n, r, l, g, max_iters), W, H, fscores, gscores, hscores, total_scores)
                plot_3d(M, W_true, W, filepath=image_filepath.format(n, r, l, g, max_iters))
                
                with open(csv_filename, 'a', newline='') as csv_file:
                    csv_writer = csv.writer(csv_file)
                    csv_writer.writerow(['Proximal averaging with penalty', n, r, l, g, len(total_scores), time_taken, save_filepath.format(n, r, l, g, max_iters), image_filepath.format(n, r, l, g, max_iters)])


In [7]:
save_filepath = '../saved_models/multi_synthetic/prox_avg/n{}_r{}_l{}_g{}_it{}.npz'
image_filepath = '../images/multi_synthetic/prox_avg/n{}_r{}_l{}_g{}_it{}.jpg'

for n in [10, 100, 1000, 10000]:

    if os.path.exists(data_filepath.format(n)):
        data = np.load(data_filepath.format(n))
        M = data['M']
        W_true = data['W_true']
        H_true = data['H_true']
    else:
        M, W_true, H_true = create_synthetic_data(n)
        with open(data_filepath.format(n), 'wb') as fout:
            np.savez_compressed(fout, M=M, W_true=W_true, H_true=H_true)

    m, n = M.shape
    if os.path.exists(ini_filepath.format(n, r)):
        data = np.load(ini_filepath.format(n, r))
        ini_W = data['ini_W']
        ini_H = data['ini_H']
    else:
        ini_W = np.random.rand(m, r) * 5
        ini_H = np.random.rand(r, n)
        with open(ini_filepath.format(n, r), 'wb') as fout:
            np.savez_compressed(fout, ini_W=ini_W, ini_H=ini_H)


    for g in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]:
        for l in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]:
            start_time = time.time()
            W, H, fscores, gscores, total_scores = sonnmf_old(M, ini_W.copy(), ini_H.copy(), lam=l, w_update_method='proximal_averaging', itermax=max_iters, W_update_iters=10, early_stop=True, verbose=False)
            time_taken = time.time() - start_time

            save_results_old(save_filepath.format(n, r, l, g, max_iters), W, H, fscores, gscores, hscores, total_scores)
            plot_3d(M, W_true, W, filepath=image_filepath.format(n, r, l, g, max_iters))
            
            with open(csv_filename, 'a', newline='') as csv_file:
                csv_writer = csv.writer(csv_file)
                csv_writer.writerow(['Proximal averaging', n, r, l, g, len(total_scores), time_taken, save_filepath.format(n, r, l, g, max_iters), image_filepath.format(n, r, l, g, max_iters)])


Early stopping condition reached at iteration 2902.


NameError: name 'hscores' is not defined

In [ ]:
save_filepath = '../saved_models/multi_synthetic/admm/n{}_r{}_l{}_g{}_it{}.npz'
image_filepath = '../images/multi_synthetic/admm/n{}_r{}_l{}_g{}_it{}.jpg'

for n in [10, 100, 1000, 10000]:

    if os.path.exists(data_filepath.format(n)):
        data = np.load(data_filepath.format(n))
        M = data['M']
        W_true = data['W_true']
        H_true = data['H_true']
    else:
        M, W_true, H_true = create_synthetic_data(n)
        with open(data_filepath.format(n), 'wb') as fout:
            np.savez_compressed(fout, M=M, W_true=W_true, H_true=H_true)

    m, n = M.shape
    if os.path.exists(ini_filepath.format(n, r)):
        data = np.load(ini_filepath.format(n, r))
        ini_W = data['ini_W']
        ini_H = data['ini_H']
    else:
        ini_W = np.random.rand(m, r) * 5
        ini_H = np.random.rand(r, n)
        with open(ini_filepath.format(n, r), 'wb') as fout:
            np.savez_compressed(fout, ini_W=ini_W, ini_H=ini_H)


    for g in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]:
        for l in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]:
            start_time = time.time()
            W, H, fscores, gscores, hscores, total_scores = sonnmf_old(M, ini_W.copy(), ini_H.copy(), lam=l, w_update_method='admm', itermax=max_iters, W_update_iters=10, early_stop=True, verbose=False)
            time_taken = time.time() - start_time

            save_results(save_filepath.format(n, r, l, g, max_iters), W, H, fscores, gscores, hscores, total_scores)
            plot_3d(M, W_true, W, filepath=image_filepath.format(n, r, l, g, max_iters))
            
            with open(csv_filename, 'a', newline='') as csv_file:
                csv_writer = csv.writer(csv_file)
                csv_writer.writerow(['ADMM', n, r, l, g, len(total_scores), time_taken, save_filepath.format(n, r, l, g, max_iters), image_filepath.format(n, r, l, g, max_iters)])
